<a href="https://colab.research.google.com/github/bestKUFO/ApiCrawling/blob/main/api%EA%B5%AC%ED%98%84_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 주소 호출


In [ ]:
#!pip install requests

In [1]:
import requests
import re
import pandas as pd

#URL 작동 형식 => Baseurl + Access_Token + Version + Action + Target + SearchQuery + SortField + CurPage + str(page) + RowCount
Baseurl = "https://apigateway.kisti.re.kr/openapicall.do?client_id="
Access_Token = "&token=" # 2시간마다 갱신되는 토큰
Version = "&version=1.0"
Action = "&action=search" # API액션 구분
Target = "&target=ARTI" #논문 약어
field = "{\"BI\":"
search = "\"메타버스\"}"
SearchQuery = "&searchQuery="+field+search # 검색필드 항목 : 검색어
SortField = "&sortField=title" #정렬 필드
CurPage = "&curPage=" #현재 페이지, cur*row = 10,000건 미만으로 제한
row_count = 10 #한 페이지당 출력 건수
RowCount = "&rowCount="+str(row_count) #디스플레이 건수 기본값:10, 최대값: 100

row_num = []
article_title = []
article_abstract = []

논문 내용 호출 및 저장

In [2]:
#정해진 페이지 탐구하기
class Article:
  def __init__(self, max_page_num):
    self.max_page_num = max_page_num

  def getURL(self): # url을 만든 후 텍스트화 시킨 뒤 논문명과 초록을 각각 추출해 리스트에 저장하는 것을 반복
    page = 1
    
    for page in range(1,self.max_page_num+1):
      url = Baseurl + Access_Token + Version + Action + Target + SearchQuery + SortField + CurPage + str(page) + RowCount

      response = requests.get(url)  
      page_text = response.text
      

      for r in range(1,row_count+1):
        #논문 번호 추가
        page_row_num = f"{page}_{r}"
        row_num.append(page_row_num)

        #타이틀 호출 및 리스트에 저장
        startidx = page_text.find("논문명")
        page_text = page_text[startidx+14:]
        lastidx = page_text.find('</item>')
        article_title.append(page_text[:lastidx-3])

        #초록 호출 및 리스트에 저장
        startidx = page_text.find("초록")
        page_text = page_text[startidx+13:]
        lastidx = page_text.find('</item>')
        article_abstract.append(page_text[:lastidx-3])
      
      page += 1


    # print(response)
    # print(url)

  def callAll(self): # 리스트에 저장한 모든 문서 불러옴
    for a in range(len(row_num)):
      print(row_num[a], article_title[a], article_abstract[a], sep="\n")

  def analysis_title(self):
    #논문 제목 분석
    str_title = " ".join(article_title) # 문자열로 변환
    str_title_upper = str_title.upper() # 대문자로 변환
    str_title_upper = re.sub("[《》:<>‘’]"," ",str_title_upper) #특수기호 제거
    split_title = str_title_upper.split() # split으로 단어로 나눠줌
    title_count={}
    for i in split_title:
      try: title_count[i] += 1 
      except: title_count[i] = 1
    
    print(title_count)
  
  def analysis_abstract(self):
    #논문 초록 분석
    str_abstract = " ".join(article_abstract)
    str_abstract_upper1 = str_abstract.upper()
    str_abstract_upper = re.sub("[《》:<>‘’]"," ",str_abstract_upper1)
    split_abstract = str_abstract_upper.split()
    select_abstract = []
    abstract_count={}
    for i in split_abstract:
      try: abstract_count[i] += 1
      except: abstract_count[i] = 1    
    
    print(abstract_count)



In [3]:
#논문 검색 최대 페이지(페이지*페이지당 검색 갯수 <= 10,000 ), 한 페이지당 검색 갯수(기본값 10, 최대 100)
art = Article(3)
art.getURL()

#중복값 제거
set_article_title = set(article_title)
article_title = list(set_article_title)
set_article_abstract = set(article_abstract)
article_abstract = list(set_article_abstract)

In [ ]:
# arti_df = None
arti_df = pd.DataFrame({"Num":row_num,"Title":article_title,"Abstract":article_abstract})
# print(arti_df['Title'])
# print(arti_df['Abstract'])
arti_df.to_csv("논문.csv", encoding="utf-8-sig")


In [ ]:
# 번외) pickle로 데이터 받기
# import pickle

# test_df = pd.read_pickle("논문")

# test_df

형태소 분석을 하기위해 khaii를 불러와줌


In [ ]:
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python

판다스에 저장된 데이터를 불러와 형태소 분석

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

#리스트로 변환하기
data_lst = arti_df["Abstract"]
# data_list = []
# for lst in data_lst:
#   data_list += lst

# print(data_lst)

#string으로 변환
# data = " ".join(data_list)
# # 형태소분석

#명사로된 리스트 Noun_list


# 테스트 결과 출력 
for word in data_lst:
  analyzed = api.analyze(word)
  # print(word.morphs[0].lex)
  print(analyzed[0].morphs[0].lex)